In [1]:
from Controller import TrainingController, ExplainingController
from Utils.SaveUtils import load_parameters
from Parameters import TrainingParameters, PredictingParameters
import tensorflow as tf
from IPython.core.display import display, HTML
import json
import numpy as np
import pandas as pd
from Parameters.Enums import TracePermutationStrategies
from Models import DiCEBinaryOutputModelWithResource
from itertools import chain

import dice_ml
from dice_ml.utils import helpers

print('TF version: ', tf.__version__)
print('Eager execution enabled: ', tf.executing_eagerly()) # False
seed = 123
tf.random.set_seed(seed)
np.random.seed(seed)

TF version:  2.4.0-rc0
Eager execution enabled:  True


In [2]:
folder_path = "./SavedModels/%s" % (
    "0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443" # AOW
)

In [3]:
parameters_json = load_parameters(folder_path=folder_path)
parameters = TrainingParameters(**parameters_json)
tf.random.set_seed(parameters.dataset_split_seed)
np.random.seed(parameters.dataset_split_seed)
parameters.load_model_folder_path = folder_path
predicting_parameters = PredictingParameters()
predicting_parameters.load_model_folder_path = folder_path
 

In [4]:
trainer = TrainingController(parameters = parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Preprocessed data loaded successfully: ./datasets/preprocessed/BPI_Challenge_2012_with_resource/AOW 

| Model loaded successfully from: ./SavedModels/0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443  


In [5]:
explainer = ExplainingController(parameters=parameters, predicting_parameters= predicting_parameters)


| Running on /job:localhost/replica:0/task:0/device:CPU:0  

| Model loaded successfully from: ./SavedModels/0.8264_BPI2012WithResource_BaselineLSTMWithResource_2021-06-18 06:11:10.009443  


In [6]:
########### Get example data from trainer ###########
ordered_test_idx = (list(trainer.test_dataset.unbatch().as_numpy_iterator()))
ordered_test_idx.sort()
print("Test set length: %d" %(len(ordered_test_idx)))

Test set length: 1309


In [7]:
caseids, example_data, example_lengths, example_resources, example_amount, _ = trainer.dataset.collate_fn([ordered_test_idx[4]])

In [8]:
remove_trail_steps = 3
example_idx_trace = np.array([example_data[0][:-remove_trail_steps]])
example_idx_resources = np.array([example_resources[0][:-remove_trail_steps]])
example_vocab_trace = trainer.model.vocab.list_of_index_to_vocab_2d(example_idx_trace)[0]

In [9]:
example_vocab_trace # A_DECLINED_COMPLETE

['<SOS>',
 'A_SUBMITTED_COMPLETE',
 'A_PARTLYSUBMITTED_COMPLETE',
 'A_PREACCEPTED_COMPLETE',
 'W_Afhandelen leads_COMPLETE',
 'A_ACCEPTED_COMPLETE',
 'O_SELECTED_COMPLETE',
 'A_FINALIZED_COMPLETE',
 'O_CREATED_COMPLETE',
 'O_SENT_COMPLETE',
 'W_Completeren aanvraag_COMPLETE',
 'W_Nabellen offertes_COMPLETE']

In [10]:
example_input = tf.constant(example_idx_trace)
example_resources_input = tf.constant(example_idx_resources)
example_amount_input = tf.constant(example_amount)

In [11]:
out, _ = explainer.model(example_input, input_resources = example_resources_input, amount = example_amount_input)

In [12]:
predicted_vocab_distributions = tf.gather(out, len(example_vocab_trace)-1, axis=1)
predicted_vocab_distributions_df = pd.DataFrame(predicted_vocab_distributions.numpy().tolist(),columns=explainer.model.vocab.vocabs)
max_arg = tf.math.argmax(predicted_vocab_distributions, axis=-1).numpy()[0]
max_prob_vocab = explainer.model.vocab.index_to_vocab(max_arg)
print("Predicted activity with highest probability (%.2f) is \"%s\"" % (predicted_vocab_distributions[0][max_arg].numpy(), max_prob_vocab))
display(predicted_vocab_distributions_df)
print(predicted_vocab_distributions_df.iloc[0])

Predicted activity with highest probability (0.75) is "W_Nabellen offertes_COMPLETE"


,<PAD>,<EOS>,<SOS>,A_ACCEPTED_COMPLETE,A_ACTIVATED_COMPLETE,A_APPROVED_COMPLETE,A_CANCELLED_COMPLETE,A_DECLINED_COMPLETE,A_FINALIZED_COMPLETE,A_PARTLYSUBMITTED_COMPLETE,...,O_DECLINED_COMPLETE,O_SELECTED_COMPLETE,O_SENT_BACK_COMPLETE,O_SENT_COMPLETE,W_Afhandelen leads_COMPLETE,W_Beoordelen fraude_COMPLETE,W_Completeren aanvraag_COMPLETE,W_Nabellen incomplete dossiers_COMPLETE,W_Nabellen offertes_COMPLETE,W_Valideren aanvraag_COMPLETE
0,0.000004,0.000358,0.000002,0.000031,0.000037,0.000167,0.020353,0.002481,0.00001,0.000019,...,0.000748,0.015224,0.153527,0.000005,0.000047,0.000017,0.00011,0.000018,0.753731,0.00007


<PAD>                                      0.000004
<EOS>                                      0.000358
<SOS>                                      0.000002
A_ACCEPTED_COMPLETE                        0.000031
A_ACTIVATED_COMPLETE                       0.000037
A_APPROVED_COMPLETE                        0.000167
A_CANCELLED_COMPLETE                       0.020353
A_DECLINED_COMPLETE                        0.002481
A_FINALIZED_COMPLETE                       0.000010
A_PARTLYSUBMITTED_COMPLETE                 0.000019
A_PREACCEPTED_COMPLETE                     0.000052
A_REGISTERED_COMPLETE                      0.000045
A_SUBMITTED_COMPLETE                       0.000039
O_ACCEPTED_COMPLETE                        0.000105
O_CANCELLED_COMPLETE                       0.052788
O_CREATED_COMPLETE                         0.000012
O_DECLINED_COMPLETE                        0.000748
O_SELECTED_COMPLETE                        0.015224
O_SENT_BACK_COMPLETE                       0.153527
O_SENT_COMPL

In [13]:
indexes_to_remove_from_resource = [explainer.resources.index('<PAD>'), explainer.resources.index('<SOS>'),explainer.resources.index('<EOS>')]

In [14]:
example_idx_trace_without_tags = [ i for i in  example_idx_trace[0] if not i in explainer.vocab.tags_idx()]
example_vocab_trace_without_tags = explainer.model.vocab.list_of_index_to_vocab(example_idx_trace_without_tags)

example_idx_resources_without_tags = [ i for i in  example_idx_resources[0] if not i in indexes_to_remove_from_resource ]

example_vocab_resource_without_tags = [
   explainer.resources[r]  for r in example_idx_resources_without_tags
]

print("=============Example activities without tags=============")
print(example_vocab_trace_without_tags)

print("=============Example resources without tags=============")
print(example_vocab_resource_without_tags)

=============Example activities without tags=============
['A_SUBMITTED_COMPLETE', 'A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE', 'W_Afhandelen leads_COMPLETE', 'A_ACCEPTED_COMPLETE', 'O_SELECTED_COMPLETE', 'A_FINALIZED_COMPLETE', 'O_CREATED_COMPLETE', 'O_SENT_COMPLETE', 'W_Completeren aanvraag_COMPLETE', 'W_Nabellen offertes_COMPLETE']
=============Example resources without tags=============
['112', '112', '11001', '11001', '11120', '11120', '11120', '11120', '11120', 'UNKNOWN', 'UNKNOWN']


In [15]:
activity_feature_names = np.array(["step_%d_activity" % (i+1) for i in range(len(example_vocab_trace_without_tags))])
without_tags_vocabs = [ v for v in explainer.vocab.vocabs if not "<" in v]
activity_cat_vars_without_tag = {}
for k in activity_feature_names:
    activity_cat_vars_without_tag[k] = without_tags_vocabs

resource_feature_names = np.array(["step_%d_resource" % (i+1) for i in range(len(example_vocab_trace_without_tags))])
without_tags_resources = [r for r in explainer.resources if not "<" in r ]
resources_cat_vars_without_tag = {}
for k in resource_feature_names:
    resources_cat_vars_without_tag[k] = without_tags_resources

In [16]:
desired_activity = 'A_APPROVED_COMPLETE'
print("Desired activity is \"%s\"" %(desired_activity))

Desired activity is "A_APPROVED_COMPLETE"


In [17]:
all_features_and_range = {**activity_cat_vars_without_tag, **resources_cat_vars_without_tag,"amount": [0, 99999.0]}

In [18]:
dice_binary_model = DiCEBinaryOutputModelWithResource(explainer.model, explainer.vocab, desired=explainer.vocab.vocab_to_index(desired_activity),trace_length = len(example_vocab_trace_without_tags), without_tags_vocabs = without_tags_vocabs, without_tags_resources=without_tags_resources, resources= explainer.resources, sos_idx_activity=explainer.vocab.vocab_to_index("<SOS>"), sos_idx_resource= explainer.resources.index('<SOS>'), amount_min = all_features_and_range['amount'][0], amount_max = all_features_and_range['amount'][1])

In [19]:
d = dice_ml.Data(features=all_features_and_range, outcome_name="predicted",continuous_features = ['amount'])
m = dice_ml.Model(model=dice_binary_model, backend="TF2")
exp = dice_ml.Dice(d, m)

In [20]:
feature_names = activity_feature_names.tolist() + resource_feature_names.tolist() + ['amount']

In [21]:
query_instance = [example_vocab_trace_without_tags + example_vocab_resource_without_tags + example_amount]
example_df = pd.DataFrame(query_instance, columns= feature_names)

In [22]:
dice_exp = exp.generate_counterfactuals(
        example_df,
        total_CFs=1,
        verbose=True,
        min_iter=100,
        max_iter=600,
        desired_class="opposite",
        features_to_vary="all",
        algorithm = "DiverseCF", # DiverseCF, RandomInitCF
        proximity_weight=0.1, #0.5,
        diversity_weight=50000,#1.0,
        init_near_query_instance=True,
        tie_random = True,
        categorical_penalty = 1,
        learning_rate=0.0005,
    )

INFO:root: MAD is not given for feature amount, so using 1.0 as MAD instead.
step 1,  loss=9.70618
step 51,  loss=9.69539
step 101,  loss=9.68461
step 151,  loss=9.6846
step 201,  loss=9.6846
step 251,  loss=9.6846
step 301,  loss=9.6846
step 351,  loss=9.6846
step 401,  loss=9.6846
step 451,  loss=9.6846
step 501,  loss=9.6846
step 551,  loss=9.6846
No Counterfactuals found for the given configuation, perhaps try with different  values of proximity (or diversity) weights or learning rate... ; total time taken: 03 min 21 sec


In [27]:
#### Check what permutations are fed into the model. ####
all_received_activities = []
for i in range(len(example_vocab_trace_without_tags)):
    trace = explainer.model.vocab.list_of_index_to_vocab(list(set((np.squeeze(np.array(dice_binary_model.all_trace))[:, i]).tolist())))
    all_received_activities.append(trace)
print(all_received_activities)
 

[['A_SUBMITTED_COMPLETE'], ['A_PARTLYSUBMITTED_COMPLETE', 'A_PREACCEPTED_COMPLETE'], ['A_PREACCEPTED_COMPLETE'], ['O_SELECTED_COMPLETE', 'W_Afhandelen leads_COMPLETE'], ['A_PREACCEPTED_COMPLETE', 'A_ACCEPTED_COMPLETE'], ['O_SELECTED_COMPLETE', 'A_SUBMITTED_COMPLETE'], ['A_FINALIZED_COMPLETE', 'O_CANCELLED_COMPLETE'], ['A_CANCELLED_COMPLETE', 'O_CREATED_COMPLETE'], ['O_SENT_COMPLETE', 'O_CANCELLED_COMPLETE'], ['O_DECLINED_COMPLETE', 'W_Completeren aanvraag_COMPLETE'], ['W_Nabellen offertes_COMPLETE', 'A_SUBMITTED_COMPLETE']]


In [26]:
#### Check what permutations are fed into the model. ####
all_received_resources = []
for i in range(len(example_vocab_trace_without_tags)):
    res = [ explainer.resources[r] for r in list(set((np.squeeze(np.array(dice_binary_model.all_resource))[:, i]).tolist()))]
    all_received_resources.append(res)
print(all_received_resources)

[['10789', '112'], ['112', '11200'], ['10779', '11001'], ['11001', '10913'], ['11120', '11121'], ['10972', '11120'], ['11120', '11289'], ['10859', '11120'], ['11120', '11201'], ['11302', 'UNKNOWN'], ['10881', 'UNKNOWN']]


In [ ]:
########## Checking if all the permutation are the same (Shouldn't be) ##########
all_cf_are_same = all([all((dice_binary_model.all_cf_input[1] == dice_binary_model.all_cf_input[i]).tolist()[0][:, 1:]) for i in range(len(dice_bi
nary_model.all_cf_input))][1:])
print("All inputs are the same: %s" % (all_cf_are_same))

In [ ]:
########## Checking if all the trace & resource (except amount) are the same (Shouldn't be) ##########
all_trace_and_resource_are_same = all([all((dice_binary_model.all_cf_input[1][:, 1:] == dice_binary_model.all_cf_input[i][:, 1:]).tolist()[0]) for
 i in range(len(dice_binary_model.all_cf_input))][1:])
print("All trace and resource are the same: %s" % (all_trace_and_resource_are_same))